<a href="https://colab.research.google.com/github/Mosminic/sapt-automation/blob/main/Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

def format_pytorch_version(version):
    return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
    return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric  -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html


! pip install git+https://github.com/divelab/DIG.git

! pip install pymatgen

# import statements
from dig.threedgraph.method import SphereNet
from dig.threedgraph.evaluation import ThreeDEvaluator
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pandas as pd
from dig.threedgraph.method import run
from pymatgen.core.structure import Molecule

Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.9/884.9 kB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=5dd22c453c4c33cb1dcca9666682eeec6f246cfbe22b0ebc367f13f435f41ca0
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric
  Cloning https://github.com/divelab/DIG.git to /tmp/pip-req-build-zd0aehpc
 

In [ ]:
from pandas.core.arrays.period import validate_dtype_freq
# This is to remove the folder dataset if it already exists.
# Useful when you want to rerun the whole code

! rm -fr *_dataset

! mkdir -p train_dataset/dimers/raw
! mkdir -p val_dataset/dimers/raw
! mkdir -p test_dataset/dimers/raw

#train, validate, and test csv files need to be made
all_calculations = pd.read_csv("SAPT DATA 5000.csv")
#print(all_calculations.head())
shuffled = all_calculations.sample(frac=1)
#print(shuffled.head())

In [ ]:
updatedDF = pd.DataFrame()
for i in range(len(shuffled)):
  xyz_lines = shuffled["xyz"][i].split('\\n')
  xyz_multiline = '\n'.join(xyz_lines)
  #print(xyz_multiline)
  with open ("dimer.xyz","w") as dimer_file:
    dimer_file.write(xyz_multiline)

  # Read the molecule from XYZ
  dimer = Molecule.from_file("dimer.xyz")

  # We need to separate the two molecules to mol_A and mol_B to ge ZA, ZB and RA, RB
  num_atoms_molecule = int(len(dimer.sites) / 2)
  mol_A = Molecule.from_sites(dimer.sites[0:num_atoms_molecule])
  mol_B = Molecule.from_sites(dimer.sites[num_atoms_molecule:])

  # data
  ZA = np.array(mol_A.atomic_numbers)
  ZB = np.array(mol_B.atomic_numbers)
  RA = mol_A.cart_coords
  RB = mol_B.cart_coords
  #to dictionary
  shufdict = shuffled.loc[i].to_dict()
  shufdict.update({"ZA": ZA})
  shufdict.update({"ZB": ZB})
  shufdict.update({"RA": RA})
  shufdict.update({"RB": RB})
  updatedDF = updatedDF.append(shufdict, ignore_index = True)

Streaming output truncated to the last 5000 lines.
<ipython-input-6-7e7d8cc33623>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updatedDF = updatedDF.append(shufdict, ignore_index = True)
<ipython-input-6-7e7d8cc33623>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updatedDF = updatedDF.append(shufdict, ignore_index = True)
<ipython-input-6-7e7d8cc33623>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updatedDF = updatedDF.append(shufdict, ignore_index = True)
<ipython-input-6-7e7d8cc33623>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updatedDF = updatedDF.append(shufdict, ignore_index = True)
<ipython-input-6-7e7d8cc33623

In [ ]:
train_index = int(0.6 * len(shuffled))
validate_index = int(0.2 * len(shuffled))
#test_num = int(0.2 * len(shuffled))

train_df = updatedDF[0:train_index]
validate_df = updatedDF[train_index:(train_index + validate_index)]
test_df = updatedDF[(train_index + validate_index):]
updatedDF.head()

In [ ]:
train_df.to_pickle("train_dataset/dimers/raw/dimers.pkl")
validate_df.to_pickle("val_dataset/dimers/raw/dimers.pkl")
test_df.to_pickle("test_dataset/dimers/raw/dimers.pkl")

In [ ]:
import os.path as osp
import numpy as np
from tqdm import tqdm
import torch
from sklearn.utils import shuffle
import pandas as pd

from torch_geometric.data import InMemoryDataset, download_url
from torch_geometric.data import Data, DataLoader


class Dimer(InMemoryDataset):

    def __init__(self, root='dataset/', transform=None, pre_transform=None, pre_filter=None):

        self.folder = osp.join(root, 'dimers')

        super(Dimer, self).__init__(self.folder, transform, pre_transform, pre_filter)

        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return 'dimers.pkl'

    @property
    def processed_file_names(self):
        return 'dimers_processed.pt'


    def process(self):
        print("Loading dataset",flush=True)
        dataset = pd.read_pickle(osp.join(self.raw_dir, self.raw_file_names))
        dataset.dropna(inplace=True)
        print("Loaded dataset",flush=True)


        data_list = []
        print(" Adding data ",flush=True)
        for idx,rows in tqdm(dataset.iterrows()):
            # These are the atomic coordinates
            R_i = torch.tensor(np.concatenate([rows["RA"],rows["RB"]]), dtype=torch.float32)

            # The atomic number
            z_i = torch.tensor(np.concatenate([rows["ZA"],rows["ZB"]]), dtype=torch.int64)

            # The label for training the model
            y_i = [torch.tensor(rows[name], dtype=torch.float32) for name in
                   ["Electrostatics",	"Exchange",	"Induction",	"Dispersion"]]

            data = Data(pos=R_i, z=z_i, y=y_i[0], elst=y_i[0],	exch=y_i[1],	ind=y_i[2],	disp=y_i[3])

            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]
        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)

        print('Saving...',flush=True)
        torch.save((data, slices), self.processed_paths[0])

    def get_idx_split(self, data_size, train_size, valid_size, seed):
        ids = shuffle(range(data_size), random_state=seed)
        train_idx, val_idx, test_idx = torch.tensor(ids[:train_size]), torch.tensor(
            ids[train_size:train_size + valid_size]), torch.tensor(ids[train_size + valid_size:])
        split_dict = {'train': train_idx, 'valid': val_idx, 'test': test_idx}
        return split_dict

In [ ]:
train_dataset = Dimer(root="train_dataset")
valid_dataset = Dimer(root="val_dataset")
test_dataset = Dimer(root="test_dataset")

Loading dataset
Loaded dataset
 Adding data 


Processing...
2820it [00:00, 4342.59it/s]


Saving...
Loading dataset
Loaded dataset
 Adding data 


Done!
Processing...
940it [00:00, 4670.55it/s]

Saving...
Loading dataset
Loaded dataset
 Adding data 



Done!
Processing...
941it [00:00, 4451.32it/s]

Saving...



Done!


In [ ]:
# Set the target for prediction
target = "ind"
train_dataset.data.y = train_dataset.data[target]
valid_dataset.data.y = valid_dataset.data[target]
test_dataset.data.y = test_dataset.data[target]

model = SphereNet(energy_and_force=False, cutoff=5.0, num_layers=4,
        hidden_channels=128, out_channels=1, int_emb_size=64,
        basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,
        num_spherical=3, num_radial=6, envelope_exponent=5,
        num_before_skip=1, num_after_skip=2, num_output_layers=3, use_node_features=True
        )
loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

device = torch.device('cuda:0')
run3d = run()
run3d.run(device, train_dataset, valid_dataset, test_dataset, model, loss_func, evaluation, epochs=50, batch_size=16, vt_batch_size=16, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=15, log_dir=target+"/logs", save_dir=target)


/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


KeyboardInterrupt: ignored

In [ ]:
# Set the target for prediction
target = "exch"
train_dataset.data.y = train_dataset.data[target]
valid_dataset.data.y = valid_dataset.data[target]
test_dataset.data.y = test_dataset.data[target]

model = SphereNet(energy_and_force=False, cutoff=5.0, num_layers=4,
        hidden_channels=128, out_channels=1, int_emb_size=64,
        basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,
        num_spherical=3, num_radial=6, envelope_exponent=5,
        num_before_skip=1, num_after_skip=2, num_output_layers=3, use_node_features=True
        )
loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

device = torch.device('cuda:0')
run3d = run()
run3d.run(device, train_dataset, valid_dataset, test_dataset, model, loss_func, evaluation, epochs=50, batch_size=16, vt_batch_size=16, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=15, log_dir=target+"/logs", save_dir=target)


In [ ]:
# Set the target for prediction
target = "disp"
train_dataset.data.y = train_dataset.data[target]
valid_dataset.data.y = valid_dataset.data[target]
test_dataset.data.y = test_dataset.data[target]

model = SphereNet(energy_and_force=False, cutoff=5.0, num_layers=4,
        hidden_channels=128, out_channels=1, int_emb_size=64,
        basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,
        num_spherical=3, num_radial=6, envelope_exponent=5,
        num_before_skip=1, num_after_skip=2, num_output_layers=3, use_node_features=True
        )
loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

device = torch.device('cuda:0')
run3d = run()
run3d.run(device, train_dataset, valid_dataset, test_dataset, model, loss_func, evaluation, epochs=50, batch_size=16, vt_batch_size=16, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=15, log_dir=target+"/logs", save_dir=target)


/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


#Params: 1890118

=====Epoch 1

Training...


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
100%|██████████| 244/244 [01:07<00:00,  3.61it/s]



Evaluating...



100%|██████████| 82/82 [00:13<00:00,  6.00it/s]

...



Evaluating...



100%|██████████| 82/82 [00:13<00:00,  6.17it/s]



Testing...



100%|██████████| 82/82 [00:12<00:00,  6.61it/s]


{'Train': 0.28483361530987944, 'Validation': 0.6454351544380188, 'Test': 0.5548166632652283}
Best validation MAE so far: 0.6170984506607056
Test MAE when got best validation result: 0.5783576965332031


In [ ]:
# Set the target for prediction
target = "elst"
train_dataset.data.y = train_dataset.data[target]
valid_dataset.data.y = valid_dataset.data[target]
test_dataset.data.y = test_dataset.data[target]

model = SphereNet(energy_and_force=False, cutoff=5.0, num_layers=4,
        hidden_channels=128, out_channels=1, int_emb_size=64,
        basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,
        num_spherical=3, num_radial=6, envelope_exponent=5,
        num_before_skip=1, num_after_skip=2, num_output_layers=3, use_node_features=True
        )
loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

device = torch.device('cuda:0')
run3d = run()
run3d.run(device, train_dataset, valid_dataset, test_dataset, model, loss_func, evaluation, epochs=50, batch_size=16, vt_batch_size=16, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=15, log_dir=target+"/logs", save_dir=target)


#Predictions


In [ ]:
def predict(batch, checkpoint):
    # Here we create the model object
    model = SphereNet(energy_and_force=False, cutoff=5.0, num_layers=4,
            hidden_channels=128, out_channels=1, int_emb_size=64,
            basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,
            num_spherical=3, num_radial=6, envelope_exponent=5,
            num_before_skip=1, num_after_skip=2, num_output_layers=3, use_node_features=True
            )

    # As we have already trained the model, we can load the weights and biases from the checkpoint file
    model.load_state_dict(torch.load(checkpoint,map_location=torch.device('cuda:0'))["model_state_dict"])
    model.to("cuda:0")
    # We set the model to evaluation mode as we don't want the model to learn now
    model.eval()

    # This loops over multiple inputs if provied
    results = []
    for data in batch:
        data.to("cuda:0")
        # Here we get one prediction
        prediction = model(data)

        # prediction is of type tensor so we convert it to array
        results.append(round(prediction.cpu().detach().numpy()[-1][0],3))
    return results

def predict_from_molecule(molecule, checkpoint):
    """
    args:

    molecule (Molecule): pymatgen Molecule object
    checkpoint (str): path to checkpoint file

    Description:

    Makes prediction from Molecule object
    """
    R_i = torch.tensor(molecule.cart_coords, dtype=torch.float32)
    z_i = torch.tensor(np.array(molecule.atomic_numbers), dtype=torch.int64)
    data = Data(pos=R_i, z=z_i,)
    batch = DataLoader([data], batch_size=1)
    return predict(batch,checkpoint=checkpoint)

def predict_from_file(filename, checkpoint):
    """
    args:

    filename (str): path to xyz file
    checkpoint (str): path to checkpoint file

    Description:

    Makes prediction from XYZ file with dimer coordinates
    """
    molecule = Molecule.from_file(filename)
    return predict_from_molecule(molecule, checkpoint)


In [ ]:
xyz_location = "/content/PENTACENE-1-DIMER.xyz"

elst = predict_from_file(xyz_location,"/content/elst.pt")
ind = predict_from_file(xyz_location,"/content/ind.pt")
disp = predict_from_file(xyz_location,"/content/disp.pt")
exch = predict_from_file(xyz_location,"/content/exch.pt")
total_energy = elst[0] + ind[0] + disp[0] + exch[0]

In [ ]:
print("Electrostatic Energy: " + str(elst[0]))
print("Induction Energy: " + str(ind[0]))
print("Dispersion Energy: " + str(disp[0]))
print("Exchange Energy: " + str(exch[0]))
print("Total Energy: " + str(total_energy))

Predictions with a csv file of dimers

In [ ]:
predictions_df = pd.read_csv("SAPT Predictions 5000.csv")
predictions_pd = pd.DataFrame()
for i in range(len(predictions_df)):
  dimer_file = open("dimer.xyz", "w")
  dimer_file.write(predictions_df["xyz"][i])
  dimer_file.close()

  # Read the molecule from XYZ
  dimer = Molecule.from_file("dimer.xyz")

  # We need to separate the two molecules to mol_A and mol_B to ge ZA, ZB and RA, RB
  num_atoms_molecule = int(len(dimer.sites) / 2)
  mol_A = Molecule.from_sites(dimer.sites[0:num_atoms_molecule])
  mol_B = Molecule.from_sites(dimer.sites[num_atoms_molecule:])

  # data
  ZA = np.array(mol_A.atomic_numbers)
  ZB = np.array(mol_B.atomic_numbers)
  RA = mol_A.cart_coords
  RB = mol_B.cart_coords
  #to dictionary
  pdict = predictions_df.loc[i].to_dict()
  pdict.update({"ZA": ZA})
  pdict.update({"ZB": ZB})
  pdict.update({"RA": RA})
  pdict.update({"RB": RB})
  predictions_pd = predictions_pd.append(pdict, ignore_index = True)

<ipython-input-4-eee742afd328>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_pd = predictions_pd.append(pdict, ignore_index = True)
<ipython-input-4-eee742afd328>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_pd = predictions_pd.append(pdict, ignore_index = True)
<ipython-input-4-eee742afd328>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_pd = predictions_pd.append(pdict, ignore_index = True)
<ipython-input-4-eee742afd328>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_pd = predictions_pd.append(pdict, ignore_index = True)
<ipython-input-4-eee742afd328>:27: FutureWarning: Th

In [ ]:
predictions_pd["Electrostatic Energy Prediction"] = 0
predictions_pd["Induction Energy Prediction"] = 0
predictions_pd["Dispersion Energy Prediction"] = 0
predictions_pd["Exchange Energy Prediction"] = 0
predictions_pd["Total Energy Prediction"] = 0

In [ ]:
for idx, row in predictions_pd.iterrows():
  dimer_file = open("dimer.xyz", "w")
  dimer_file.write(predictions_pd["xyz"][idx])
  dimer_file.close()
  xyz_location = "/content/dimer.xyz"
  elst = predict_from_file(xyz_location,"/content/elst-5000.pt")
  ind = predict_from_file(xyz_location,"/content/ind-5000.pt")
  disp = predict_from_file(xyz_location,"/content/disp-5000.pt")
  exch = predict_from_file(xyz_location,"/content/exch-5000.pt")
  total_energy = elst[0] + ind[0] + disp[0] + exch[0]

  predictions_pd["Electrostatic Energy Prediction"][idx] = elst[0]
  predictions_pd["Induction Energy Prediction"][idx] = ind  [0]
  predictions_pd["Dispersion Energy Prediction"][idx] = disp[0]
  predictions_pd["Exchange Energy Prediction"][idx] = exch[0]
  predictions_pd["Total Energy Prediction"][idx] = total_energy

  print(idx)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
<ipython-input-6-71fc4c50e358>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions_pd["Electrostatic Energy Prediction"][idx] = elst[0]
<ipython-input-6-71fc4c50e358>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions_pd["Induction Energy Prediction"][idx] = ind  [0]
<ipython-input-6-71fc4c50e358>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
predictions_pd.iloc[1]

Dispersion                                                                     -17.8
Electrostatics                                                                  -8.5
Exchange                                                                       19.12
Induction                                                                      -2.23
Total                                                                          -9.41
_id                                                                 csd_XEZDIP0_1_41
identifier                                                                csd_XEZDIP
label                                                                         0_1_41
xyz                                100\nH38 C52 N2 O8\nH 5.432012 7.086142 3.5330...
ZA                                 [1, 7, 6, 6, 6, 8, 6, 8, 6, 6, 6, 6, 6, 6, 6, ...
ZB                                 [1, 7, 6, 6, 6, 8, 6, 8, 6, 6, 6, 6, 6, 6, 6, ...
RA                                 [[5.432012, 7.086142, 3.533034

In [ ]:
predictions_pd.head()
predictions_pd.to_csv("SAPTML Results - 3.csv")

In [ ]:
pdf = pd.DataFrame()
pdf["xyz"] = predictions_pd["xyz"]

pdf["Electrostatic"] = predictions_pd["Electrostatics"]
pdf["Electrostatic Prediction"] = predictions_pd["Electrostatic Energy Prediction"]
pdf["Elst Difference"] = predictions_pd["Electrostatics"] - predictions_pd["Electrostatic Energy Prediction"]
pdf["Elst Percent Error"] = pdf["Elst Difference"] / pdf["Electrostatic"]

pdf["Dispersion"] = predictions_pd["Dispersion"]
pdf["Dispersion Prediction"] = predictions_pd["Dispersion Energy Prediction"]
pdf["Disp Difference"] = predictions_pd["Dispersion"] - predictions_pd["Dispersion Energy Prediction"]
pdf["Disp Percent Error"] = pdf["Disp Difference"] / pdf["Dispersion"]

pdf["Induction"] = predictions_pd["Induction"]
pdf["Induction Prediction"] = predictions_pd["Induction Energy Prediction"]
pdf["Ind Difference"] = predictions_pd["Induction"] - predictions_pd["Induction Energy Prediction"]
pdf["Ind Percent Error"] = pdf["Ind Difference"] / pdf["Induction"]

pdf["Exchange"] = predictions_pd["Exchange"]
pdf["Exchange Prediction"] = predictions_pd["Exchange Energy Prediction"]
pdf["Exch Difference"] = predictions_pd["Exchange"] - predictions_pd["Exchange Energy Prediction"]
pdf["Exch Percent Error"] = pdf["Exch Difference"] / pdf["Exchange"]

pdf["Total"] = predictions_pd["Total"]
pdf["Total Prediction"] = predictions_pd["Total Energy Prediction"]
pdf["Total Difference"] = predictions_pd["Total"] - predictions_pd["Total Energy Prediction"]
pdf["Total Percent Error"] = pdf["Total Difference"] / pdf["Total"]

In [ ]:
pdf.head()
pdf.to_csv("extra SAPTML Results - 1.csv")